In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection  import train_test_split

In [12]:
class RegressionLogistique():
    
    def __init__(self , learning_rate = 0.01 , nbIter = 1000):
        self.learning_rate = learning_rate
        self.nbIter = nbIter
        self.w = None
        self.b0 = None
        
    def _sigmoid(self ,x):
        return 1/(1+np.exp(-x))

        
    # pour apprendre le modele
    def train(self,  data , target):
        if(data.shape[0] != target.shape[0]):
            raise ValueError(" les données doivent avoir la meme taille")
        n_observation ,n_feature = data.shape # le nombre d'observation et de feature        
        
        self.w = np.zeros(n_feature)
        self.b0 = 0
        
        #puisque minimiser la fonction de cout de la fonction logistique n'est pas analytiquement facile ,
        #nous allons utilisee  la descente de gradiant simple
        
        for _ in range(self.nbIter):
            #f(x) = WtX + b
            # Ypre = 1/(1+ exp(-fx))
            f_x  = np.dot( data ,self.w ) + self.b0
            Ypre = self._sigmoid(f_x);
            
            #gradiant de g = 1/n * SUM(y - ypre) * xi
            dw = 1/n_observation*(np.dot(data.T , (target - Ypre))) 
            db0 = 1/n_observation * (np.sum((target - Ypre)))
            
            #Mise à jour 
            
            self.w -= self.learning_rate*dw
            self.b0 -= self.learning_rate *db0
        print(self.w , "le biais" ,self.b0 )
        
    def inference(self , x_test):
            #Yi =   1 /(1+exp(-(Wx +b)))
       # print(np.dot( x_test ,  self.w ) + self.b0)
        ypredict = self._sigmoid(np.dot( x_test ,  self.w ) + self.b0)
        for i in range(len(ypredict)) : 
            if ypredict[i] >= 0.5:
                ypredict[i] =1
            else:
                ypredict[i] = 0
        print(ypredict[:30])
        return ypredict
        

In [4]:
spams = pd.read_csv("./spam.csv", header = 0, encoding = 'ISO-8859-14', sep = ',' )


In [5]:
spams.describe()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [6]:
spams = spams[['v1', 'v2']] #on recupere les 2 colonnes les plus importantes
spams.columns = ['class', 'text'] #on renomme
spams.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
from sklearn.feature_extraction.text import  CountVectorizer

# on instancie le converteur de donnees texte de sklearn
data_converter = CountVectorizer()
v = data_converter.fit(spams.text)
# on realise la conversion
spam_convert = data_converter.fit_transform(spams.text)
data = spam_convert.toarray()

In [13]:

def binarisation(test_labels):
    target= []
    for label in test_labels :
        if label =="ham":
            target.append(0)
        else:
            target.append(1)
    return np.array(target)

target= binarisation(spams["text"])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split( data, target, train_size=0.6, test_size=0.4, stratify= target)
RL = RegressionLogistique()
RL.train(x_train , y_train)
RL.inference(x_test)